# 3.4 Cleaning Data

Cleaning data is one of pandas' primary functions. It is often said that data scientists spend anywhere from 40-50% of their time cleaning data, a necessary step before analysis can occur.

### What is "dirty data"?

Many rows of data are often useless to the overall data analysis because they (1) *contain null values*, (2) *do not match the datatype* of other values in the column, (3) because the *format is inconsistent with other, similar values*, or (4) because the data contains extreme or unexpected values. Data is called "dirty" when values like this appear in the dataset.

##### Contains null values
Data containing null values can be problematic because they can skew analysis. If you have a dataset that is 100 rows long, for example, but only three of those rows have a value for the "Age" column, the `.mean()` calculation will likely not be representative of the entire population. There are several ways to deal with null values, although we will not go over them extensively in this notebook. Many data scientists use **imputation** (calculate and assign a value to each row) to fill in the blanks, and others simply **drop** the null rows or the entire column. The decision of "what to do" really depends on the situation and the context of the data.

In the event that very few values are available in the data set, it may be best to just drop the column, since there may be no way to know what the values should be. If only a few rows are missing values, you might decide to fill them in with either the most frequently occurring value or the average/median value.

##### Data type doesn't match
If the data types in a column don't match, there could be problems with analysis. For example, in a dataset that records the number of bathrooms in a house, one row could list a house with an integer (1) and another house with a string ("two"). The values `1` and `"two"` are different data types. They either both need to be converted to be the same data type, or one needs to be removed from the data set.

##### Inconsistent format
Imagine that you are performing aggregations on a dataset and grouping by U.S. state. However, some of the values for "state" are recorded using the two-letter code (ie. "UT") and others are recorded with the full name of the state ("Utah"). Pandas wouldn't know that these two places should be grouped together and so each one will be its own group. Thus, these values need to be changed so that they are uniform.

##### Extreme/unexpected/incorrect values
Sometimes, due to errors in the data collection process, values can be saved incorrectly to a dataset. These values may *technically* meet the requirements of the database; however, they logically don't make sense. 

For example, in a dataset about personal income where many rows describe yearly income in amounts greater than \\$1,000, it would be unexpected to see a value of `1`. It might also be unexpected to see a negative number. As data analysts, we need to decide how to interpret and fix these rows. For example, we might simply be able to change the negative number to a positive number and be done, but because we can't know if `1` means \\$1,000 or \\$1,000,000, we might decide to simply drop this row.

Other examples of extreme/unexpected values would be a birth date that is in the future; letter grades that are not A, B, C, D, or F; and inventory amounts that are negative.

Errors that are simply incorrect are difficult to detect-- however, finding them and fixing them is an essential step in the data cleaning process!

### About the data
​
The data used in this notebook shows information about passengers on the *Titanic* cruiseliner, a ship which set out from Southampton, U.K. to sail across the Atlantic ocean and which tragically sank upon collision with an iceberg. The dataset contains information about each passenger's passenger class, name, sex, age, siblings, parents/children, ticket number, ticket fare, cabin number, and the embarked location. It also contains information about each passenger's survival status. This data set is extremely popular among data scientists and will facilitate demonstrations of Pandas concepts.

In [ ]:
import pandas as pd
df = pd.read_csv("./data/titanic.csv")
df.head()

### Data contains null values
Pandas has several methods for looking at null values. Note that in Pandas, null values are called `NaN` (not a number).

##### `.isna()`
The `.isna()` method can be applied to a Series or a dataframe and will return a boolean value describing if the value was missing (null) or not.

In [ ]:
df.isna()

We can count up the number of null values in each column by adding the `.sum()` aggregation method at the end.

In [ ]:
df.isna().sum()

When it comes to null values, there are several things we could do. If the data is categorical and there are few null values, we might simply assign the most common category to each null row. If the data is quantitative, we might assign each null value the average or median of the column. This is called *imputation*.

Another approach would be to drop rows with null values. This would reduce the overall number of observations but could remove some inacurracies that could be produced by imputation.

Let's **impute** the two missing values for the "Embarked" column by assigning those two values the most commonly occuring value. We can do this by getting the most common value with the `.mode()` method and then using the `.fillna()` method to fill in each null value with the mode.

Note that the `.mode()` method always returns a Series (since their can be multiple modes). In this case, we'll just get the first mode by adding `[0]` after `.mode()`.

In [ ]:
most_common_embarked = df['Embarked'].mode()[0] # The `.mode()` method returns a Series-- get the first and only item in the Series
df['Embarked'].fillna(most_common_embarked, inplace=True)

In [ ]:
df.isna().sum()

It looks like there are 177 null values in the "Age" column. That seems like a lot to impute since there are only 800 ish rows in total. Let's **drop** them instead, with the `.dropna()` method. This method accepts a parameter `how`, which can be "any" or "all", meaning that it will drop the row if *any* of the columns in the subset are null or only if *all* columns in the subset are null. The `subset` parameter requires a list of columns to look for null values in.

We will also use the `inplace` argument to update the original dataframe.

In [ ]:
df.dropna(how='any', subset=['Age'], inplace=True)

Now let's look at the total number of null values again.

In [ ]:
df.isna().sum()

### Data type doesn't match

We can use the `.dtypes` property to see the data types of each column as automatically interpreted by Pandas. Note that data type `object` usually means "string".

In [ ]:
df.dtypes

Observe that the "Age" column has the data type `float64`, meaning that each person's age is represented as a decimal number. That might be intentional, but if we wanted to group passengers by their age we might want them to express their age as an integer instead. 

We can convert the values in the column to an integer type using the `.astype()` method. To use this method, we simply select a Series, use the `.astype()` method on it, and pass in a new data type as a string. The most common data types are `float64`, `int64`, and `object`. The `.astype()` method returns a new Series, so we need to save it on top of the existing `Age` column.

In [ ]:
df['Age'] = df['Age'].astype('int64')

Changing the data type "cleans up" the data a little bit by rounding the decimal numbers to whole numbers. That said, in this example, changing `Age` from a `float64` to an `int64` type probably won't make a huge difference in our analysis. However, there may be instances when changing the data type will make sense for aggregating and also processing efficiency.

For example, in the example above, we changed the `Age` column to be of data type `int64`. By definition, the `int64` data type takes up 64 bits of memory in the computer hardware (that's 64 0's and 1's to store each number). If you had a large data set, you might prefer to store the data in less bits to take up less space for more efficient processing, using data types like `int32`, `int16`, or even `int8`, especially since ages don't tend to go above 255 (the largest number that can be stored in 8 bits). In other words, you don't need very many bits to store numbers like ages, so you can store them in smaller memory chunks and not have drawbacks.

In [ ]:
df.dtypes

### Inconsistent format
Sometimes information contained in the columns of a data set is correct, but the values are formatted differently. For example, the `Survived` column might have used 1's to indicate that a passenger survived, but also may have used the word "True" and the letter "T" to indicate the same thing. "True", "T", and 1 would all be accurate and correct descriptions of the passengers' survival, but the format would not allow the data to be processed and aggregated correctly.

In this data set, it appears that tickets have no standardization. Some of them start with numbers and some of them start with letters. All of them, however, seem to have a string of five or six numbers at the end. We can trim off the leading letters and preserve just the string of numbers by applying a function to the column.

In [ ]:
df.head()

To begin creating our function, let's remember what the `.apply()` method does. The `.apply()` method is used on a Series and accepts a function name (ie. *without parentheses*). This function should accept only one parameter which will represent each value in the Series (ie. represents a single row). The function will then do something to that parameter (ie. do something to each row individually) and return each row. The `.apply()` method thus executes the function on each row of the Series and returns the results in a new Series.

In this example, we will define a new named function called `getTicketNumber()` that accepts one parameter that we will call `rawTicket`. This represents each ticket number in the `Ticket` column individually (since we applied the function to the column `Ticket`).

Because I checked the data type of the `Ticket` column previously with the `.dtypes` property, I know that it contains a bunch of strings. Thus, I know that I can apply the `.split()` method to the `rawTicket`, which is a string method. The `.split()` method splits a string into a list of strings using a set of characters that you give it. In the example below, we split on a space `" "`. That means that everything to the left of a space character will be put into a list, and everything to the right will be placed into the same list as the second item. If there were no spaces in the `rawTicket`, then a list with just one item is returned.

This list of strings is then saved as `ticketParts`.

We then check the length of the list. If the list only has one item (ie. `len(ticketParts) == 1`) then the ticket didn't have spaces, so we can just return the first and only item `[0]`. Otherwise (if the list *did* have a space), we will return the last item `[-1]`.

We then pass the name of our function into the `.apply()` method and save the result as a new column `TicketNumber`.

In [ ]:
def getTicketNumber(rawTicket):
    ticketParts = rawTicket.split(" ") # split the ticket by its spaces into a list
    if len(ticketParts) == 1:          # If the list only has one value (ie. no spaces in ticket, so wasn't split)
        return ticketParts[0]          # Return the first and only value
    else:                              # Otherwise return the last item in list (gets number without preceding text)
        return ticketParts[-1]
    
df['TicketNumber'] = df['Ticket'].apply(getTicketNumber)

Let's check to make sure that our function worked, and that the column was added correctly.

In [ ]:
df.head()

### Extreme/unexpected/incorrect values
Finding data with extreme or unexpected values can be challenging. This is because software (like Pandas) is only concious of whether or not datatypes are consistent in each column, but doesn't know the logic behind the column. Thus, for example, while paying a negative Fare amount to get on the Titanic might be obviously suspicious to us humans, a computer will just assume that it is normal. As a result, it's up to the data analyst to find mistakes in the data and correct them.

There isn't a specific Pandas function for fixing unexpected values. That is because the solution for fixing unexpected values differs across many situations. In some instances, you might just need to fix a single cell in a specific row, which you could do using the `.loc` property. In other cases, you might need to fix an entire column of values, which you could do by using either the `.loc` property or the `.apply()` method in combination with a named function or a lambda function.

In the example below, I want to show you how you might locate extreme or incorrect data while doing analysis. Then, I'll show you in an example how I might fix the data using the `.loc` property.

#### Finding unexpected values
If we conduct a quick `df.info()`, we can see how many non-null rows we have and what the data types of each column are. It might be useful to run this function sometimes to look for anomalies. In this case, however, it doesn't tell us very much, especially since we've already cleaned the data somewhat.

In [ ]:
df.info()

We can also use `df.describe()` to see some summaries about our data. What do you notice about the fields below? Do any of the summary statistics jump out to you?

In [ ]:
df.describe()

Notice the minimum value in the `SibSp` column. It's `-1`! If we don't know what data the `SibSp` is supposed to have, we might not recognize the problem right away. If we were to investigate a little more, we would find out that `SibSp` shows the number of siblings and spouses that each passenger has on board the *Titanic*. Knowing that, it's obvious that `-1` is an impossible value to exist, even though Pandas didn't throw an error or complain.

Let's assume that the value should actually just be positive 1 and turn each -1 into positive 1. We can find each occurrence where `SibSp = -1` by using a filter. 

In [ ]:
df.loc[df['SibSp'] == -1]

#### Fixing the unexpected values

Now that we've found all of the rows where `SibSp = 1`, we can simply add `= 1` to the filter to turn each occurrence of `-1` into `1`. 

In [ ]:
# Turn each occurrence of -1 into 1
df.loc[df['SibSp'] == -1] = 1

Now we can describe the data again to check if the problem was corrected.

In [ ]:
df.describe()

### Final thoughts

There are other situations in which your data may need cleaning, and there are many more Pandas methods and parameters available to use. The more experienced you become with data analysis and Pandas, the better you will be able to clean your data.